In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obstructive_sleep_apnea/GSE75097'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray gene expression profiles of peripheral blood mononuclear cells (PBMC) in patients with  obstructive sleep apnea and primary snoring"
!Series_summary	"Therefore, we extended our investigation into OSA patients with long-term continuous positive airway pressure (CPAP) treatment, hypertension, or excessive daytime sleepiness (EDS) by analyzing whole-genome gene expression profiles of PBMC in three comparisons: (1) treatment-naïve moderate to very severe OSA patients versus subjects with primary snoring; (2) moderate to very severe OSA patients with hypertension or EDS versus those without hypertension or EDS, respectively; (3) treatment-naïve very severe OSA patients versus those receiving at least one year of adequate CPAP treatment."
!Series_overall_design	"We analyzed whole-genome gene expression profiles of peripheral blood mononuclear cells from 48 patients with sleep-disordered breathing stratified into four groups: primary snoring (

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Based on the series description, it mentions gene expression profiles

# Data availability determination
trait_row = 1  # Information available under 'apnea hyponea index'
age_row = 3  # Information available under 'age'
gender_row = 2  # Information available under 'Sex'

# Data type conversion functions
def convert_trait(value):
    try:
        # Extract value after the colon, convert to float, then to binary
        index_value = float(value.split(':')[-1].strip())
        return 1 if index_value > 15 else 0
    except Exception:
        return None

def convert_age(value):
    try:
        # Extract value after the colon, convert to float
        return float(value.split(':')[-1].strip())
    except Exception:
        return None

def convert_gender(value):
    try:
        # Extract value after the colon, convert gender to binary
        gender_str = value.split(':')[-1].strip().lower()
        return 1 if gender_str == 'male' else 0 if gender_str == 'female' else None
    except Exception:
        return None

save_cohort_info('GSE75097', './preprocessed/Obstructive_sleep_apnea/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obstructive_sleep_apnea', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Obstructive_sleep_apnea/trait_data/GSE75097.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1942590': [1.0, 54.0, 1.0], 'GSM1942591': [1.0, 31.0, 1.0], 'GSM1942592': [1.0, 44.0, 1.0], 'GSM1942593': [1.0, 60.0, 1.0], 'GSM1942594': [1.0, 21.0, 1.0], 'GSM1942595': [0.0, 50.0, 1.0], 'GSM1942596': [1.0, 52.0, 0.0], 'GSM1942597': [1.0, 58.0, 1.0], 'GSM1942598': [1.0, 42.0, 0.0], 'GSM1942599': [1.0, 34.0, 1.0], 'GSM1942600': [1.0, 58.0, 1.0], 'GSM1942601': [1.0, 37.0, 1.0], 'GSM1942602': [1.0, 60.0, 0.0], 'GSM1942603': [1.0, 59.0, 1.0], 'GSM1942604': [1.0, 27.0, 1.0], 'GSM1942605': [1.0, 57.0, 1.0], 'GSM1942606': [1.0, 68.0, 1.0], 'GSM1942607': [1.0, 53.0, 1.0], 'GSM1942608': [1.0, 58.0, 1.0], 'GSM1942609': [1.0, 52.0, 0.0], 'GSM1942610': [0.0, 36.0, 1.0], 'GSM1942611': [1.0, 38.0, 1.0], 'GSM1942612': [1.0, 50.0, 1.0], 'GSM1942613': [1.0, 44.0, 1.0], 'GSM1942614': [1.0, 58.0, 1.0], 'GSM1942615': [1.0, 54.0, 1.0], 'GSM1942616': [0.0, 43.0, 1.0], 'GSM1942617': [1.0, 59.0, 0.0], 'GSM1942618': [1.0, 44.0, 1.0], 'GSM1942619': [1.0, 46.0, 1.0], 'GSM1942620': [1.0, 36.0, 1.0], 'GSM194

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'ORF': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6 - Skipping as requires_gene_mapping=False: Unknown Step

In [7]:
# Proceed to the next step.


### Step 7: Data Normalization and Merging

In [8]:

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Obstructive_sleep_apnea/gene_data/GSE75097.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait 'Obstructive_sleep_apnea' and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Obstructive_sleep_apnea')

# 4. Save the cohort information.
save_cohort_info('GSE75097', './preprocessed/Obstructive_sleep_apnea/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Obstructive_sleep_apnea/GSE75097.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Obstructive_sleep_apnea', the least common label is '0.0' with 6 occurrences. This represents 12.50% of the dataset.
The distribution of the feature 'Obstructive_sleep_apnea' in this dataset is fine.

Quartiles for 'Age':
  25%: 42.0
  50% (Median): 50.0
  75%: 58.0
Min: 21.0
Max: 68.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 18.75% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Obstructive_sleep_apnea/cohort_info.json
